In [ ]:
import os

CPD_URL = "<enter your CPD endpoint here>"
CPD_USERNAME = "<username>"
CPD_PASSWORD = "<password>"

CPD_URL = CPD_URL.rstrip("/")

WOS_CREDENTIALS = {
    "url": CPD_URL,
    "username": CPD_USERNAME,
    "password": CPD_PASSWORD
}

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

PROJECT_ID = os.environ['PROJECT_ID']
CONTAINER_ID = PROJECT_ID
CONTAINER_TYPE = 'project'
EXPERIMENT_NAME = 'SciKitExperiment_fact'

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
!pip install ibm-aigov-facts-client --no-cache | tail -n 1

In [ ]:
from ibm_aigov_facts_client import AIGovFactsClient, CloudPakforDataConfig

creds=CloudPakforDataConfig(service_url=CPD_URL, username=CPD_USERNAME, password=CPD_PASSWORD, disable_ssl_verification=True)
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

Insert the data to code as a pandas dataframe. Note that the 'infer_schema' value is set to false, as we will manually set the datatypes next to avoid mixing types and confusing OpenScale's data consistency drift monitor.

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'false',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_1.head(10)


In [ ]:
data_df_1.dtypes

In [ ]:
import pandas as pd

data_df_1 = data_df_1.apply(pd.to_numeric, errors='coerce').fillna(data_df_1)
data_df_1 = data_df_1.astype({'ATTRITION': 'bool'})

In [ ]:
from sklearn.model_selection import train_test_split

X = data_df_1.drop(['ATTRITION'], axis=1)  # Features
y = data_df_1['ATTRITION']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

In [ ]:
fields=X_train.columns.tolist()
metadata_dict = {'target_col' : 'ATTRITION', 'fields':fields}

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format("employee attrition challenger"),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "ATTRITION",
    wml_client._models.ConfigurationMetaNames.CUSTOM: metadata_dict
}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
CUSTOM_ACC = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", CUSTOM_ACC)

In [ ]:
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

In [ ]:
nb_name =  "employee attrition challenger"
nb_asset_id = "c6519599-74c5-4656-908e-b39ec4e46b0f"
nb_asset_url = CPD_URL + "/analytics/notebooks/v2/" + nb_asset_id + "?projectid=" + PROJECT_ID + "&context=icp4data"

In [ ]:
latestRunId = facts_client.runs.get_current_run_id()
facts_client.runs.log_metric(latestRunId, "Custom accuracy score", CUSTOM_ACC)
facts_client.runs.set_tags(latestRunId, {"Notebook name": nb_name, "Notebook id": nb_asset_id, "Notebook URL" : nb_asset_url})
facts_client.export_facts.export_payload(latestRunId)

In [ ]:
wml_client.set.default_project(PROJECT_ID)

In [ ]:
facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

In [ ]:
print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=clf, 
    meta_props=model_props,
    training_target=['ATTRITION'],
    training_data=X)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))